<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [91]:
# Импортируем библиотеки для возможности отображения графиков Plotly через nbviewer:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio

# Библиотеки представленные в ноутбуке:
import pandas as pd
import psycopg2

# Библиотеки для парсинга данных
import requests
from bs4 import BeautifulSoup 
import urllib.parse as req

# Для подавления сообщений об ошибках при обработке pandas ответов psycopg2
import warnings
warnings.filterwarnings('ignore')


# Библиотеки для построения графиков
import plotly.express as px
import plotly.graph_objects as go

In [95]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = '*********'
USER = '*********'
PASSWORD = '*********'
HOST = '*********'
PORT = *********

In [96]:
connection = psycopg2.connect(
    dbname = DBNAME,
    user = USER,
    host = HOST,
    password = PASSWORD,
    port = PORT
)

# Юнит 3. Предварительный анализ данных

In [4]:
# Добавим простую функция для вывода результатов запросов, для сокращения кода
def exercise(answer):
    """_summary_
    Выполняем SQL запрос
    Args:
        SQL (answer): Вводим номер задания

    Returns:
        Pandas DataFrame: Возвращает ответ на запрос
"""
    return  pd.read_sql_query(answer, connection)


1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
    SELECT COUNT(*)
      FROM public.vacancies
'''

In [6]:
# результат запроса
exercise(query_3_1)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''
    SELECT COUNT(*)
      FROM public.employers
'''

In [8]:
# результат запроса
exercise(query_3_2)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса
query_3_3 = f'''
    SELECT COUNT(*)
      FROM public.areas
'''

In [10]:
# результат запроса
exercise(query_3_3)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
# текст запроса
query_3_4 = f'''
    SELECT COUNT(*)
      FROM public.industries
'''

In [12]:
# результат запроса
exercise(query_3_4)

,count
0,294


***

### Выводы по предварительному анализу данных

Предварительныый анализ данных показал, что база, состоит из **23501 работадателя** и представляет собой большую выборку вакансий **около 50 тысяч вакансий** из **294 сфер деятельности** с географией охвата вакансий **1362 региона**. Представленная в проекте база достаточна для получения хорошей видимости картины рынка работадателей со сравнением зарплат по регионам и по видам деятельности, определением загруженности рынка или дефицита кадров, анализом востребованности специалистов на рынке на момент сбора данных и с анализом навыков предъявлявленным к кандидатам.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''
    SELECT a.name AS area,
           COUNT(*) AS cnt 
      FROM public.areas AS a 
     INNER JOIN public.vacancies AS v ON a.id = v.area_id
     GROUP by a.name
     ORDER by cnt DESC
     LIMIT 5
'''

In [14]:
# результат запроса
exercise(query_4_1)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''
    SELECT COUNT(*)
      FROM public.vacancies
     WHERE salary_to IS NOT NULL
           OR salary_from IS NOT NULL
'''

In [16]:
# результат запроса
exercise(query_4_2)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''
    SELECT AVG(salary_from) AS avg_lowery , 
           AVG(salary_to) AS avg_upper
      FROM public.vacancies
'''

In [18]:
# результат запроса
round(exercise(query_4_3))

,avg_lowery,avg_upper
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [99]:
# текст запроса
query_4_4 = f'''
    SELECT schedule,
           employment,
           COUNT(*) AS count_vacancies
      FROM public.vacancies
     GROUP BY schedule,
              employment
     ORDER BY count_vacancies DESC
    OFFSET 1 
    LIMIT 1
'''

In [100]:
# результат запроса
exercise(query_4_4)

,schedule,employment,count_vacancies
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''
    SELECT experience,
           COUNT(*) AS number_of_vacancies
      FROM public.vacancies
     GROUP BY experience
     ORDER BY experience
'''

In [22]:
# результат запроса
exercise(query_4_5)

,experience,number_of_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 1 года до 3 лет,26152
3,От 3 до 6 лет,14511


***

### Выводы по детальному анализу вакансий:
- Определено количество вакансий в каждом регионе. Максимальное количество вакансий находятся в тех городах, где соответственно большая численность населения. Первое место среди городов занимает Москва с количеством **5333 вакансии** на город (*в рамках данного датасета*). В пятёрку лидеров по вакантным позициям входят: Столицы России (Москва, Санкт-Петербург), столица Беларуси - Минск, Новосибирск, и бывшая столица Казахстана - Алматы. Пятерка лидеров составляет порядка **28%** всех вакансий в исследуемом датасете. Так же стоит обратить внимания, что анализ вакансий охватывает не только территорию РФ, но и соседние государства.
- Среднее значение по предлагаемой зарплате варьируюется в диапазоне **от 71065 рублей до 110537 рублей**, при условии, что у **50%** вакансий не указана ЗП (**24073 вакансии**).
- Чаще всего работадатели ищут кандидатов на полный рабочий день и полную занятость. Данный тип рабочего графика составляет почти **72%** от всех вакансий, на **15%** кандидаты на Удалённую работу и полную занятость, **3%** кандидаты с гибким - графиком и полной занятостью, и **менее 3%** - остальные. 
- Среди требований к опыту работу, меньше всего ищут людей с опытом работы **более 6 лет**, возможно это кандидаты на более высокие должности, такие как главный инженер, директор и подобные или же какие-нибудь редкие вакансии с узкоспецилизированным родом деятельности. В наших реалиях такие вакансии по большей части выкладываются, что называется *для галочки*, и как правило они закрываются путём связей между высшим руководством и рекомендациям из других организаций. Не плохо было бы изучить данный момент для **утверждения гипотизы**. Не стоит унывать кандидатам которые только выходят на рынок труда и **не имеют опыта работы**, ведь **порядка 15%** вакансий приходится на данных кандидатов, таким образом имея фундаментальную базу знаний можно вливаться в рабочую атмосферу и набирать обороты в опыте работы, и уже на следующей ступени, где требования работы **от 1 года до 3 лет**, а это **чуть более 50%** всех вакансий(чаще всего встречающее требование по опыту работы) можно иметь хорошие шансы на желаемую работу, а не просто *набраться опыта*.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = f'''
    (SELECT e.name,
            COUNT(e.id) AS number_of_vacancies
       FROM public.vacancies AS v
       LEFT JOIN public.employers AS e ON v.employer_id = e.id
      GROUP BY e.name
      ORDER BY number_of_vacancies DESC
      LIMIT 1)

    UNION 

    (SELECT e.name,
            COUNT(e.id) AS number_of_vacancies
       FROM public.vacancies AS v
       LEFT JOIN public.employers AS e ON v.employer_id = e.id
      GROUP BY e.name
      ORDER BY number_of_vacancies DESC
     OFFSET 4
      LIMIT 1)
    
    ORDER BY number_of_vacancies DESC
'''

In [24]:
# результат запроса
exercise(query_5_1)

,name,number_of_vacancies
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [101]:
# текст запроса
query_5_2 = f'''
    SELECT a.name AS areas,
           COUNT(distinct e.id) AS employers,
           COUNT(distinct v.id) AS vacancies
      FROM public.areas AS a
      LEFT JOIN public.employers AS e ON e.area = a.id
      LEFT JOIN public.vacancies AS v ON v.area_id = a.id
     WHERE v.id IS NULL
     GROUP BY areas
     ORDER BY employers desc
     LIMIT 1
'''

In [102]:
# результат запроса
exercise(query_5_2)

,areas,employers,vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [103]:
# текст запроса
query_5_3 = f'''
    SELECT e.name,
           COUNT(DISTINCT v.area_id) AS areas_count
      FROM public.employers AS e
      LEFT JOIN public.vacancies AS v ON v.employer_id = e.id
     GROUP BY e.name
     ORDER BY areas_count DESC
     LIMIT 1
'''      

In [104]:
# результат запроса
exercise(query_5_3)

,name,areas_count
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [105]:
# текст запроса
query_5_4 = f'''
    SELECT COUNT(*)
      FROM public.employers AS e
      LEFT JOIN public.employers_industries AS ei ON e.id = ei.employer_id
     WHERE employer_id IS NULL
'''

In [106]:
# результат запроса
exercise(query_5_4)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [107]:
# текст запроса
query_5_5 = f'''
    SELECT name,
           COUNT(DISTINCT industry_id)
      FROM public.employers AS e
      LEFT JOIN public.employers_industries AS ei ON e.id = ei.employer_id
     GROUP BY name
    HAVING COUNT(DISTINCT industry_id) = 4
     ORDER BY 1
     LIMIT 1
    OFFSET 2
'''

In [108]:
# результат запроса
exercise(query_5_5)

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [109]:
# текст запроса
query_5_6 = f'''
    SELECT COUNT(*)
      FROM public.industries AS i
      LEFT JOIN public.employers_industries AS ei ON i.id = ei.industry_id
     WHERE name = 'Разработка программного обеспечения'
'''

In [110]:
# результат запроса
exercise(query_5_6)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [111]:
# код для получения списка городов-милионников
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
cities = tuple(pd.read_html(url)[1]['Город'])

In [112]:
# текст запроса
query_5_7 = f'''
    SELECT a.name,
           COUNT(v.name) AS cnt
      FROM public.vacancies AS v
      LEFT JOIN public.areas AS a ON a.id = v.area_id
      LEFT JOIN public.employers AS e ON v.employer_id = e.id
     WHERE e.name = 'Яндекс' 
           AND a.name IN {cities}
     GROUP BY a.name
    
    UNION ALL

    SELECT 'Total',
           COUNT(DISTINCT v.id)
      FROM public.vacancies AS v
      LEFT JOIN public.areas AS a ON a.id = v.area_id
      LEFT JOIN public.employers AS e ON v.employer_id = e.id
     WHERE e.name = 'Яндекс' 
           AND a.name IN {cities}
     ORDER BY 2 ASC
'''

In [113]:
# результат запроса
exercise(query_5_7)

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

### Выводы по анализу работодателей
- Выявлена пятёрка работадателей которые ищут больше всего людей на работу. На первом месте среди количества вакансий находится компания **Яндекс** которая занимает **4%** всех вакансий данного датасета (**1933 вакансии**), далее идёт **Ростелеком и Тинькофф** суммарно они занимаю **примерно 2%** анализируемых вакансий, **менее 1%** занимает **СБЕР (428 вакансий)**, закрывает пятёрку лидеров компания **Газпром нефть** с количеством **331 вакансия - 0,7% всех вакансий**.
- Анализ работадателей, которые не предлагают вакансии, показал, что в регионе Россия больше всего работадатей которые просто в которых нет вакансий. Сам регион описан странно. Возможно из-за того, что нет работы, работадатель указа своё местоположение, или это может относиться к удалённой работе, где не требуется привязка к определённому региону. Так же, указания региона Россия может сведетельствовать о том, что работадателем не правильно воспринята информация о регионе, и он указал страну, а не более конкретное местоположение вакансии. Данный критерий может вносить дизинформацию в анализ данных.
- Определено, что компания Яндекс больше всех охватывает терреторию поиска работников, а именно 181 регион, на втором месте Ростелеком 152 региона, а на 3-ем компания Спецремонт 116 регионов. Странным выглядит 4 позиция, которую занимает ИП Поляков Д.И. охватывающая 88 регионов, что для физ.лица странно, и выходит несколько вариантов: 1. Не корректные (аномальные) данные, 2. У данногой ИП есть своя организация ООО, и ему удобнее все вопросы закрывать через ИП. Просто не возможно на ИП держать такое количество сотрудников, чтобы охватывать 88 регионов! Был проведен анализ данного ИП через [Контур Фокус](https://focus.kontur.ru/). Определенно, что ИП занималось услугами по ремонтам компьютерной техники и было несколько точки по разным регионам, но найденный профиль на hh и отзывы о нём говорили о том, что организация является не реальной, а ваканции предложенные на сайте являются фикцией. Таким образом можно утвердить, что данный контрагент является следствием аномальных данных. 
- Определено, что 8419 работадателей не указывают свой род деятельности, это 1/3 всех работадателей данного датасета, это является показателем либо плохой работой по сбору данных, либо упущением работадателя, и он посчитал, что не обязательно указывать род деятельности в связи, мне кажется, что это проблема возникает на менеджере отдела кадров, который занимается размещением вакансий. Для решения данной проблемы, я бы сделал это поле обязательным для каждого работадателя, с выпадающим списком из стандартного набора рода деятельности, или как минимум с указанием [ОКВЭД](https://www.consultant.ru/document/cons_doc_LAW_163320/) (Общероссийский Классификатор Видов Экономической Деятельности). 
- Рассмотрены работадатели со сферой деятельности Разработка программного обеспечения они составляют 15 % всех работадателей (3553 работадатели)
- Анализ вакансий организации Яндекс показал, что компания обладает большим географическим охватом регионов в которых ищет сотрудников. Яндекс занимает лидирующие позицию крупнейших корпораций Рунета и имеет большой спектр услуг: такси, доставка еды, маркетплейсы, каршеринг, самокаты и т.д. Среди городов миллиоников определенно количество вакансий данной организации, это **485** вакансии. Может ли это быть связано с большой текучкой? Яндекс тщательно ведёт работу по подбору своих кадров с использованием разлиных уровней прохождения этапов собеседования. Предполагается, что в связи с тем, что организация быстрорастущая и, с увеличением географического охвата, созданием новых сервисов, точек обслуживания возникает такая потребность в кадрах. Данный датасет не раскрывает весь потенциал вакансий на сегодняшний день, но помимо того, что Яндекс работает в России есть полно вакансии и в других странах включая Беларусь, Казахстан и Турцию, а её сервисы, к примеру Яндекс GO работают и в Грузии, Молдавии, Киргизии, Сербии, Норвегии, Сенегале и в [других странах](https://ru.wikipedia.org/wiki/%D0%AF%D0%BD%D0%B4%D0%B5%D0%BA%D1%81.%D0%A2%D0%B0%D0%BA%D1%81%D0%B8). 


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [114]:
# текст запроса
query_6_1 = f'''
    SELECT COUNT(name)
      FROM public.vacancies
     WHERE name ILIKE '%данн%' 
           OR name ILIKE '%data%'
'''

In [115]:
# результат запроса
exercise(query_6_1)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [116]:
# текст запроса
query_6_2 = f'''
    SELECT COUNT(id)
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name ILIKE '%%ML%%' 
              AND name NOT ILIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%')
           AND (name ILIKE '%%junior%%'
               OR experience = 'Нет опыта'
               OR employment = 'Стажировка')
'''

In [117]:
# результат запроса
exercise(query_6_2)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [118]:
# текст запроса
query_6_3 = f'''
    SELECT COUNT(id)
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name LIKE '%%ML%%' 
              AND name NOT LIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%')
           AND (key_skills LIKE '%%SQL%%'
               OR key_skills LIKE '%%postgres%%')
'''

In [119]:
# результат запроса
exercise(query_6_3)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [120]:
# текст запроса
query_6_4 = f'''
    SELECT COUNT(key_skills)
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name LIKE '%%ML%%' 
              AND name NOT LIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%')
           AND key_skills LIKE '%%Python%%'
'''

In [121]:
# результат запроса
exercise(query_6_4)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [122]:
# текст запроса
query_6_5 = f'''
    SELECT ROUND(AVG((CHAR_LENGTH(key_skills) - CHAR_LENGTH(REPLACE(key_skills, CHR(9), ''))) + 1), 2) AS avg_ds
      FROM public.vacancies 
     WHERE (name ILIKE '%%data scientist%%'
        OR name ILIKE '%%data science%%'
        OR name ILIKE '%%исследователь данных%%'
        OR name ILIKE '%%machine learning%%'
        OR (name LIKE '%%ML%%' 
           AND name NOT LIKE '%%HTML%%')
        OR name ILIKE '%%машинн%%обучен%%')
       AND key_skills is not null
'''

In [123]:
# результат запроса
exercise(query_6_5)

,avg_ds
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [126]:
# текст запроса
query_6_6 = f'''
    SELECT experience,
           ROUND(AVG((COALESCE(salary_to, salary_from) + COALESCE(salary_from, salary_to))/2))
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name LIKE '%%ML%%' 
              AND name NOT LIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%')
          AND (salary_from IS NOT NULL 
              OR salary_to IS NOT NULL)
          AND experience = 'От 3 до 6 лет'
     GROUP BY 1
'''

In [127]:
# результат запроса
exercise(query_6_6)

,experience,round
0,От 3 до 6 лет,243115.0


### Выводы по предметному анализу

- В процессе предметного анализа выявленно, что к данным определенным к специальности Data Science относится **1771 вакансия**. (*Считалось, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'*)
- Определено, что среди всего датасета на вакансиию Data Science на позицию junior специалиста без опыта работы относится 0,1 % вакансий, а среди вакансий относящих к Data Science **2,8 %,** что весьма грустно если ты ищешь работу после обучения, а ещё с учетом большой конкурентной среды даже не хочется представлять сложность входа в данный сегмент рынка.
- Первыми из основных требований по критериям знаний к потенциальному работнику будут язык программирования Python и SQL для DS. Так SQL встречается в **11% вакансий**, а язык программирования Python **20%**. В среднем работадатели указывают порядка **6,4 ключевых навыков**.
- Но не смотря на большое количество требований и сильной конкурентной среды, специалисты в области Data Science могут рассчитывать на очень высокую зарплату **от 75 тысяч рублей до 240 тысяч** при опыте работы от 3 до 6 лет, тенденция роста зарплаты будет зависеть, как правило от большего опыта, от сложности проектов и местоположения работадателя.

***

#### Выполнение дополнительного исследования

- Исходя из выдвиннутой гипотизы по специалистам в выводах *ЮНИТА 4. Детальный анализ вакансий*, проведем дополнительную проработку: выполним запрос, и рассмотрим, какие должности повторяются более 5 раз при поиске кандидатов на работу с *опытом работы более 6 лет**


In [52]:
# текст запроса
query_add_1 = f'''
    SELECT name,
           count(name) AS cnt
      FROM public.vacancies
     WHERE experience = 'Более 6 лет'
     GROUP BY name
    HAVING count(name) > 5

UNION ALL

    SELECT 'Остальные',
            sum(cnt)
      FROM (
           SELECT name, 
                  count(name) AS cnt
             FROM public.vacancies
            WHERE experience = 'Более 6 лет'
            GROUP BY name
           HAVING count(name) < 5
            ) AS others
     ORDER BY 2 ASC
'''
exercise(query_add_1)

,name,cnt
0,Архитектор решений с опытом в Tiny ML,6.0
1,Заместитель технического директора по автомати...,6.0
2,Бизнес-аналитик,6.0
3,Ведущий разработчик 1С,6.0
4,Главный инженер производства,7.0
5,Руководитель направления Data analyst,7.0
6,Senior System Engineer (Admins Team),7.0
7,Начальник отдела информационных технологий,7.0
8,Senior System Administrator,8.0
9,Разработчик 1С,8.0


* Наша гипотиза (*возможно это кандидаты на более высокие должности*) в большей части подтверждена. Как видно из запроса чаще всего встречаются должности с опытом работы более 6 лет на позиции **директоров, руководителей, Senior** а также **ведущих специалистов** в разных областях деятельности. Все **Остальные** имеют единичные случая при поиске кандидатов, скорее всего это узконаправленные специалисты, или обычные специлисты, но к которым выдвинуты специфические требования или большое их количество

- Опустив рамки с опытом работы, посмотрим на сколько востребованна на рынке специалисты по DataScience среди остальных вакансий. Выведим топ 10 профессий:

In [128]:
# текст запроса
query_add_2 = f'''
    (SELECT name,
            COUNT(id) as cnt
       FROM public.vacancies
      GROUP BY 1
      ORDER BY cnt desc
      LIMIT 10)

UNION ALL

     (SELECT 'Data Science',
              COUNT(distinct id) as cnt
       FROM public.vacancies
      WHERE (name ILIKE '%%data scientist%%'
            OR name ILIKE '%%data science%%'
            OR name ILIKE '%%исследователь данных%%'
            OR (name ILIKE '%%ML%%' 
               AND name NOT ILIKE '%%HTML%%')
            OR name ILIKE '%%machine learning%%'
            OR name ILIKE '%%машинн%%обучен%%'))
ORDER BY cnt desc
            
'''

# Строим диаграмму
fig = px.bar(
    exercise(query_add_2),
    x = 'name',
    y = 'cnt',
    color = 'name',
    width=1200,
    height=700
)

# Настройка осей графика:
fig.update_layout(
    title={
        'text': "График востребованности специалистов на рынке", # наименование легенды
        'font': {'size': 24}, # размер шрифта
        'x' : .5 # Расположение по центру
    },
    xaxis={
        'title': {
            'text': 'Название вакансии', # наименование оси
            'font': {'size': 20}  # размер шрифта наименования оси
        },
        'tickfont': {'size': 13}  # размер шрифта название данных на оси
    },
    yaxis={
        'title': {
            'text': 'Количество вакансий, шт', # наименование оси
            'font': {'size': 20}  # размер шрифта наименования оси
        },
        'tickfont': {'size': 13}, # размер шрифта название данных на оси
    },
    margin=dict(l=40, r=40, t=40, b=40), # Настройка отступов
    legend_title_text = 'Вакансии' # Меняем название легенды
)

# Настройка информации при наведении курсора:
fig.update_traces(
    hovertemplate="Вакансий: %{y} анкет", # Информация при наведении курсора на колонку
    marker_line_color='black',   # Цвет линии колонки
    marker_line_width=2.5,       # Толщина линии колонки
    opacity=0.75            # Прозрачность колонки
)

fig.show()


- График востребованности специалистов на рынке показал, что специалисты по Data Science являются востребованными, и занимают 3 строчку по количеству анкет на данное направление. Но стоит, иметь ввиду, что профессии с анализом данных со временем не будут терять актуальности в ближайшие 10-15 лет, так как данных становится больше, их нужно всё больше обрабатывать и использовать в промышленности, в бизнесе. Соответсветстввенно востребованность и зарплата на Data Science будет расти. Так к примеру, Российский рынок данных стабильно растёт с 2010 года, и продолжает свой рост. Карта исскуственного интелекта не останавливается в развитии и своём росте. На данном сайте [Карта исскуственного интеллекта России](http://airussia.online/#titul) можно рассмотреть все компании, которые пользуются ИИ, географию их распределения по городам, данные по росту рынка  и т.д. что подтвердает направление движения заинтересованности работадателей в специалистах по Data Science

- Рассмотрим среднии зарплатные пороги по профессии Data Science в зависимости от опыта работы:

In [129]:
# текст запроса
query_add_3 = f'''
    SELECT experience,
           AVG(salary_from) AS avg_lowery,
           AVG(salary_to) AS avg_upper
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name ILIKE '%%ML%%' 
              AND name NOT ILIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%')
     GROUP BY experience
     ORDER BY experience
'''
round(exercise(query_add_3))


,experience,avg_lowery,avg_upper
0,Более 6 лет,157933.0,NaN
1,Нет опыта,61000.0,97000.0
2,От 1 года до 3 лет,125119.0,175566.0
3,От 3 до 6 лет,220401.0,308393.0


- Из полученного запроса видно, что с ростом стажа растёт и уровень зарабатной платы. Однако для специалистов со стажем более 6 лет, не указанны высшие средниее данные по зарплате (avg_upper).  Скорее всего с таким большим стажем трудно оценить специалиста, и в большенстве своём размер выплат будет иметь договорной характер, т.е. за сколько готова купить компания данного специалиста, и на какую сумму специалист сам себя оценивает

- Для того, чтобы вывести ориентиры по направлению Data Science, выведим данные по 10 чаще встречаемым требованиям в вакансиях. И построим график по популярности в процентном соотношении среди 10 требований.
В запросе SQL используется функция разделения строки `regexp_split_to_table`, для преобразования требований(**key_skill**) из списка в строку по позиционно. Подробнее о функции `regexp_split_to_table` можно прочитать [здесь](https://postgrespro.ru/docs/postgresql/9.6/functions-matching).

In [130]:
# Строим запрос по данным 10 популярных требований к DS
query_add_4 = f'''
    SELECT 
        regexp_split_to_table(key_skills, chr(9)) as skill,
        count(*) as skill_count
      FROM public.vacancies
     WHERE (name ILIKE '%%data scientist%%'
           OR name ILIKE '%%data science%%'
           OR name ILIKE '%%исследователь данных%%'
           OR (name ILIKE '%%ML%%' 
              AND name NOT ILIKE '%%HTML%%')
           OR name ILIKE '%%machine learning%%'
           OR name ILIKE '%%машинн%%обучен%%'
           AND key_skills is not null)
     GROUP BY skill
     ORDER BY skill_count desc
     LIMIT 10
'''

# Строим круговой график:

fig = go.Figure(
    data=[go.Pie(
    values=exercise(query_add_4)['skill_count'], 
    pull=[0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05],
    labels=exercise(query_add_4).skill)]
)

# Задаем цвета:

fig.update_traces(
    hoverinfo='value+label',
    textinfo='percent',
    textfont_size=20,
    marker=dict(line=dict(color='#000000', width=2))
)

# Устанавливаем название графика и размеры текста:
fig.update_layout(
    title_text="Процентное соотношение 10 самых популярных требований к Data Science",
    title_font_size=20,
    legend_font_size=20,
    width = 1000,
    height=800
)

fig.show()

- Исходя из диаграммы процентного соотношения 10 самых популярных требований к Data Science, определенно, что в первую очередь работадатель обращает внимание на знание языка программирования Python, SQL, владение Machine Learning, Git и мат. статистику. Думаю, что для начинающих специалистов без опыта работы стоит обратить на эти ключевые навыки и прорабатывать их как можно лучше, чтобы быть конкурентноспособным на рынке труда. 

# Общий вывод по проекту

#### В процессе проведения проекта выполнен:
1. Предварительный анализ данных;
1. Детальный анализ вакансий;
1. Анализ работадателей;
1. Предметный анализ;
1. Дополнительный анализ;

#### Определено что:
- Текущий датасет содержит около 50 тысяч вакансий с 23,5 тысячью работадателей из 294 сферы деятельности с географией охватата 1,3 тысячи региона;
- Среднии зарплаты по зарплатам варьируются от 71 тысячи рублей до 110 тысячи рублей;
- Чаще всего работадатели ищут кандидатов на полный рабочий день с полной занятостью и опытом работы от 1 года до 3х лет.
- Среди работадателей больше всего вакантных мест у компании Яндекс, Ростелеком и Тинькофф, суммарно они занимают 6% всех вакансий текущего датасета;
- Самым больше всего регионов задействовано у корпорации Яндекс, она охватывает 181 регион;
- Порядка 35% работадателей не указывают свой род деятельности;
- Специальность Data Science занимает 3,6 % всех вакансий датасета(1771 вакансия);
- Основными требования по ключевым навыкам являются Python, SQL, Machine Learning, Git и Математическая статистика;
- Несмотря на маленький процент вакансий среди датасета,что может показаться на первый взгляд, специалисты по Data Science более чем востребованны, и занимают 3 строчку по количеству вакансий по критерию специальности среди всех вакансий.
- Специалисты в области Data Science являются высокооплачиваемыми и их зарплата доходит до 240 тысяч рублей и выше, в зависимости от опыта работы, возможностей специалиста и от организации в которой он будет работать.

**Стоит отметить, что существует вероятность аномальных показателей в данных, созданных в процессе их сбор или при допущении ошибок со стороны их заполнения работадателем. Но сильного влияния эти ошибки не оказывают на текущий датасет и он является вполне пригодным для дальнейшего исследования.** 

In [90]:
# закрываем соединение после окончания работы над данными
connection.close()